# 02 GDP Growth and Recession Label

Compute GDP growth variants and build a technical recession label.


## Table of Contents
- [Fetch GDP](#fetch-gdp)
- [Compute growth](#compute-growth)
- [Define recession label](#define-recession-label)
- [Define next-quarter target](#define-next-quarter-target)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Data notebooks build the datasets used everywhere else. If these steps are wrong, every model result is suspect.
You will practice:
- API ingestion and caching,
- frequency alignment,
- label construction.


## What You Will Produce
- data/processed/gdp_quarterly.csv

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.
- You can point to the concrete deliverable(s) listed below and explain how they were produced.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Merging mixed-frequency series without explicit resampling/aggregation.
- Forgetting to shift targets for forecasting tasks.

## Matching Guide
- `docs/guides/01_data/02_gdp_growth_and_recession_label.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/01_data/02_gdp_growth_and_recession_label.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Compute GDP growth (multiple definitions) and define a **technical recession** label.

### Technical recession label used in this project
We define recession as:
- two consecutive quarters of negative **real GDP growth** (QoQ)

This is a teaching proxy, not an official recession dating rule.



## Primer: pandas Time Series Essentials

### DatetimeIndex
Most time series work in pandas assumes your DataFrame is indexed by time.

```python
import pandas as pd

# idx = pd.to_datetime(df['date'])
# df = df.set_index(idx).sort_index()
# assert isinstance(df.index, pd.DatetimeIndex)
```

If you see weird behavior (resample errors, merges not aligning), check:
- `df.index.dtype`
- `df.index.min(), df.index.max()`
- `df.index.is_monotonic_increasing`

### Resampling
> **Goal:** convert one frequency to another.

Common examples:
- daily -> month-end
- monthly -> quarter-end

```python
# month-end last value
# df_me_last = df.resample('ME').last()

# month-end mean
# df_me_mean = df.resample('ME').mean()

# quarter-end mean
# df_q_mean = df.resample('QE').mean()
```

**Interpretation matters.** For economic series:
- using `.last()` treats the end-of-period value as “the period’s value”
- using `.mean()` treats the period average as “the period’s value”

Neither is universally correct; you should choose based on measurement and use case.

### Alignment and merging
When joining multiple time series, you need to ensure they share:
- the same index type (`DatetimeIndex`)
- the same frequency convention (month-end vs month-start; quarter-end vs quarter-start)

```python
# Example: join two series and inspect missingness
# df = df1.join(df2, how='outer').sort_index()
# print(df.isna().sum())
```

### Lags and rolling windows
> **Lag:** use past values as features.

```python
# lag 1 period
# df['x_lag1'] = df['x'].shift(1)

# rolling mean (past-only)
# df['x_roll12'] = df['x'].rolling(12).mean()
```

### Common gotchas
- `shift(-1)` uses the future (leakage for forecasting).
- `rolling(..., center=True)` uses future values.
- Always `dropna()` after creating lags/rolls to get clean modeling rows.

One more gotcha:
- If you resample daily -> monthly and then create lags, your lag is “one month” (not one day). Lags are measured in the current index frequency.


<a id="fetch-gdp"></a>
## Fetch GDP

### Goal
Load quarterly real GDP levels (`GDPC1`).

Options:
1) Fetch from FRED (preferred if you have a key)
2) Load the offline sample (`data/sample/gdp_quarterly_sample.csv`)



### Your Turn (1): Fetch GDPC1 (or load sample)


In [ ]:
import os
import pandas as pd
from src import fred_api
from src import data as data_utils

sid = 'GDPC1'

# TODO: If you have FRED_API_KEY, fetch observations and convert to a DataFrame.
# Hint: fred_api.fetch_series_observations + fred_api.observations_to_frame
# Otherwise, load SAMPLE_DIR / 'gdp_quarterly_sample.csv'
if os.getenv('FRED_API_KEY'):
    payload = fred_api.fetch_series_observations(sid, start_date='1980-01-01', end_date=None)
    gdp = fred_api.observations_to_frame(payload, sid)
else:
    gdp = pd.read_csv(SAMPLE_DIR / 'gdp_quarterly_sample.csv', index_col=0, parse_dates=True)

gdp = gdp.sort_index()
gdp.head()



### Checkpoint (GDP shape)


In [ ]:
# TODO: Confirm quarterly-ish index and no obvious missingness.
assert isinstance(gdp.index, pd.DatetimeIndex)
assert gdp.index.is_monotonic_increasing
assert gdp.shape[1] == 1
...



<a id="compute-growth"></a>
## Compute growth

### Goal
Compute multiple growth definitions from GDP levels.

You will compute and compare:
- QoQ growth (percent)
- Annualized QoQ growth (percent)
- YoY growth (percent)



### Your Turn (1): Compute growth variants


In [ ]:
from src import macro

# GDP levels series
gdp_levels = gdp['GDPC1'].astype(float)

# TODO: Compute growth series
gdp_growth_qoq = ...
gdp_growth_qoq_ann = ...
gdp_growth_yoy = ...

gdp_feat = pd.DataFrame({
    'gdp_level': gdp_levels,
    'gdp_growth_qoq': gdp_growth_qoq,
    'gdp_growth_qoq_annualized': gdp_growth_qoq_ann,
    'gdp_growth_yoy': gdp_growth_yoy,
}).dropna()
gdp_feat.head()



### Your Turn (2): Compare the growth definitions


In [ ]:
import matplotlib.pyplot as plt

# TODO: Plot the three growth series on separate subplots.
...



<a id="define-recession-label"></a>
## Define recession label

### Goal
Construct the technical recession label from QoQ growth.

Definition used here:
- `recession_t = 1` if `growth_t < 0` AND `growth_{t-1} < 0`



### Your Turn (1): Build the label and inspect it


In [ ]:
from src import macro

# TODO: Compute technical recession label
recession = ...

# TODO: Attach to gdp_feat and inspect counts
gdp_feat['recession'] = recession
print(gdp_feat['recession'].value_counts())
gdp_feat[['gdp_growth_qoq', 'recession']].tail(12)



<a id="define-next-quarter-target"></a>
## Define next-quarter target

### Goal
Create the classifier target: **predict next quarter's recession label**.

That target is:
- `target_recession_next_q[t] = recession[t+1]`

Be careful: this creates a missing value at the end (there is no future label for the last row).



### Your Turn (1): Shift label to build the target


In [ ]:
from src import macro

# TODO: Shift recession to build next-quarter target
gdp_feat['target_recession_next_q'] = ...

# TODO: Drop rows where target is missing
gdp_feat = gdp_feat.dropna(subset=['target_recession_next_q']).copy()

gdp_feat[['recession', 'target_recession_next_q']].tail(6)



### Your Turn (2): Save to data/processed/gdp_quarterly.csv


In [ ]:
out_path = PROCESSED_DIR / 'gdp_quarterly.csv'
out_path.parent.mkdir(parents=True, exist_ok=True)

# TODO: Save gdp_feat to CSV
...



### Reflection


- Why is this only a proxy for 'recession'?
- How could this differ from official recession dating?
- How might you validate whether this label aligns with economic reality?



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
import pandas as pd

# Expected file: data/processed/gdp_quarterly.csv
# TODO: After saving your processed dataset, load it and run checks.
# df = pd.read_csv(PROCESSED_DIR / 'your_file.csv', index_col=0, parse_dates=True)
# assert df.index.is_monotonic_increasing
# assert df.shape[0] > 20
# print(df.dtypes)
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Fetch GDP</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_gdp_growth_and_recession_label — Fetch GDP
import pandas as pd
from src import data as data_utils

# Offline default
gdp = pd.read_csv(SAMPLE_DIR / 'gdp_quarterly_sample.csv', index_col=0, parse_dates=True)
gdp.head()
```

</details>

<details><summary>Solution: Compute growth</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_gdp_growth_and_recession_label — Compute growth
from src import macro

levels = gdp['GDPC1']
gdp['gdp_growth_qoq'] = macro.gdp_growth_qoq(levels)
gdp['gdp_growth_qoq_annualized'] = macro.gdp_growth_qoq_annualized(levels)
gdp['gdp_growth_yoy'] = macro.gdp_growth_yoy(levels)
```

</details>

<details><summary>Solution: Define recession label</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_gdp_growth_and_recession_label — Define recession label
gdp['recession'] = macro.technical_recession_label(gdp['gdp_growth_qoq'])
```

</details>

<details><summary>Solution: Define next-quarter target</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_gdp_growth_and_recession_label — Define next-quarter target
gdp['target_recession_next_q'] = macro.next_period_target(gdp['recession'])
gdp = gdp.dropna()
gdp.tail()
```

</details>

